In [3]:
### Python Packages
#This cell imports all the necessary Python libraries for data processing, visualization, 
#natural language processing (NLP), and machine learning. 
#It ensures all dependencies are loaded at the start for seamless notebook execution.

In [5]:
# Used Packages
import pandas as pd         # For data manipulation and handling
import numpy as np          # For numerical computations
import matplotlib.pyplot as plt  # For visualizations
import seaborn as sns       # For enhanced visualizations
from sklearn.model_selection import train_test_split  # For splitting datasets
from sklearn.feature_extraction.text import TfidfVectorizer  # For text vectorization
from sklearn.linear_model import LogisticRegression  # For training the ML model
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score  # For evaluation
import nltk                 # For natural language processing (NLP)

# Download necessary NLTK data
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/pouyasmac/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True